<a href="https://colab.research.google.com/github/pandemic-tracking/viz-gen/blob/main/notebooks/equity1a_vizgen_misc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and set themes

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import pytz

import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

from pathlib import Path

pd.set_option("display.precision", 4)

now_est = datetime.now().astimezone(pytz.timezone("US/Eastern"))

now_est_time = now_est.strftime("%Y-%m-%d, %H:%M:%S ET")
now_est_date = now_est.strftime("%Y-%m-%d")
now_est_timestamp = now_est.strftime("%Y%m%d_%H%M%S")
now_utc_timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
print(now_est_time, now_est_date, now_est_timestamp, now_utc_timestamp)

In [ ]:
# adapting from https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602

def ptc_theme():
    axisColor = "#808080"
    gridColor = "#DEDDDD"
    markColor = "#000000"
    font = 'Arial'
    labelFont = 'Arial'
    # Colors
    # main_palette = ["#1696d2", 
    #                 "#d2d2d2",
    #                 "#000000", 
    #                 "#fdbf11", 
    #                 "#ec008b", 
    #                 "#55b748", 
    #                 "#5c5859", 
    #                 "#db2b27", 
    #                ]
    # sequential_palette = ["#cfe8f3", 
    #                       "#a2d4ec", 
    #                       "#73bfe2", 
    #                       "#46abdb", 
    #                       "#1696d2", 
    #                       "#12719e", 
    #                      ]
    return {
          "config": {
              "title": {
                  "fontSize": 16,
                  "font": font,
                  "anchor": "start", # equivalent of left-aligned.
                  "fontColor": "#000000"
              },
              "text": {
                  "font": font,
                  "labelFont": labelFont,
              },
              "header": {
                  "font": font,
                  "labelFont": labelFont,
                  "titleFont": font,
              },
              "axisX": {
                  "domain": False,
                  "domainColor": axisColor,
                  "labelColor": axisColor,
                  "domainWidth": 1,
                  "grid": False,
                  "labelFont": labelFont,
                  "labelFontSize": 12,
                  "labelAngle": 0, 
                  "tickColor": axisColor,
                  "tickSize": 5, # default, including it just to show you can change it
                  "titleFont": font,
                  "titleFontSize": 12,
                  "titlePadding": 10, # guessing, not specified in styleguide
                  "title": "X Axis Title (units)", 
              },
              "axisY": {
                  "domain": False,
                  "grid": True,
                  "gridColor": gridColor,
                  "gridWidth": 1,
                  "labelFont": labelFont,
                  "labelColor": axisColor,
                  "labelFontSize": 12,
                  "labelAngle": 0,
                  "labelAnchor": "end",
                  "labelAlign": "right",  
                  "ticks": False, # even if you don't have a "domain" you need to turn these off.
                  "titleFont": font,
                  "titleFontSize": 12,
                  "titlePadding": 10, # guessing, not specified in styleguide
                  "title": "Y Axis Title (units)", 
                  # titles are by default vertical left of axis so we need to hack this 
                  "titleAngle": 0, # horizontal
                  "titleY": -10, # move it up
                  "titleX": 18, # move it to the right so it aligns with the labels 
              },
            #   "range": {
            #       "category": main_palette,
            #       "diverging": sequential_palette,
            #   },
              "legend": {
                  "labelFont": labelFont,
                  "labelFontSize": 12,
                  "symbolType": "circle", # just 'cause
                  "symbolSize": 100, # default
                  "titleFont": font,
                  "titleFontSize": 12,
                  "title": "", # set it to no-title by default
                  "orient": "right", # so it's right next to the y-axis
                  "offset": 0, # literally right next to the y-axis.
              },
              "view": {
                  "stroke": "transparent", # altair uses gridlines to box the area where the data is visualized. This takes that off.
              },
        }
    }

alt.themes.register("my_custom_theme", ptc_theme)
alt.themes.enable("my_custom_theme")

# Get input data

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime
import sys
import pytz

"""Federal data combiner tool. Downloads federal testing, hospitalization, and case/death data and combines it
into unified state-level output. 

This script outputs a csv file named federal-covid-data-DATE.csv and is intended to be run in the Google Colab 
environment, but passing the 'STDOUT' argument will output to STDOUT instead for command-line use.

Note: We at The COVID Tracking Project will not be maintaining this script in the event that the federal data pages 
change in impactful ways. This is simply a set of instructions for interested data users.

Changelog: 
2021-04-28: Added new age-disaggregated hospital admission data columns
2021-03-15: Initial release
"""

HHS_TESTING_URL = "https://www.healthdata.gov/api/views/j8mb-icvb/rows.csv?accessType=DOWNLOAD"
HHS_HOSPITALIZATION_URL = "https://www.healthdata.gov/api/views/g62h-syeh/rows.csv?accessType=DOWNLOAD"
HHS_HOSPITALIZATION_REVISIONS_URL = "https://www.healthdata.gov/api/views/4cnb-m4rz/rows.csv?accessType=DOWNLOAD"
CDC_CASE_DEATH_URL = "https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD"

COMBINE_NY_NYC = True


def get_hospitalization_csv_urls():
    """get a set of recent revisions for the HHS hospitalizations-by-state dataset"""
    revisions = pd.read_csv(HHS_HOSPITALIZATION_REVISIONS_URL)

    revisions['Update Date'] = pd.to_datetime(revisions['Update Date'], format='%m/%d/%Y %H:%M:%S %p')
    return revisions.sort_values(by=['Update Date']).tail(n=8)['Archive Link'].tolist()


def get_hospitalization_dailies():
    """build a dataframe containing the combination of several days of recent HHS hospitalization daily data"""
    csv_urls = get_hospitalization_csv_urls()
    data_frames = []

    for url in csv_urls:
        data = pd.read_csv(url, parse_dates=['reporting_cutoff_start'])
        # set each daily file's date to the reporting_cutoff_start date + 4 days
        data['date'] = data['reporting_cutoff_start'] + pd.DateOffset(days=4)
        data['date'] = data['date'].dt.date
        data_frames.append(data)
    hospitalization_dailies = pd.concat(data_frames)
    return hospitalization_dailies


def parse_dates(date_col):
  if date_col.str.contains(":").any():
    return pd.to_datetime(date_col, format='%m/%d/%Y %H:%M:%S %p').dt.date
  else: 
    return pd.to_datetime(date_col, format='%Y-%m-%d').dt.date
  

# download and parse all three data files
[testing, hospitalization, case_death] = [pd.read_csv(url) for url in
                                          [HHS_TESTING_URL, HHS_HOSPITALIZATION_URL, CDC_CASE_DEATH_URL]]

# testing data comes out with one row per state/date/outcome combination.
# unpack that and squash it into one row per state/date only
testing = testing.set_index(['state', 'date', 'overall_outcome']).unstack(level=-1)
testing = testing[['new_results_reported', 'total_results_reported']]
testing.columns = ['_'.join(tup).rstrip('_') for tup in testing.columns.values]
testing = testing.reset_index()
# allow for a choice of inconsistent datetime formats
testing['date'] = parse_dates(testing['date'])

# the HHS hospitalization time series is only updated weekly. To compensate, we download the latest daily data
# and merge it on top of the weekly data, taking only the most recent values for a given state/date
hospitalization['date'] = parse_dates(hospitalization['date'])
hospitalization_dailies = get_hospitalization_dailies()
# we want to use the HHS weekly time series up until its last day, then fill in the rest of the data from the daily
# files. we overwrite the last day of the time series with the dailies because the dailies come out after the weekly
hospitalization_dailies = hospitalization_dailies[hospitalization_dailies['date'] >= hospitalization['date'].max()]
hospitalization.set_index(['state', 'date'])
# don't break everything with missing values in geocoded_state
hospitalization_dailies['geocoded_state'] = hospitalization_dailies['geocoded_state'].fillna("")
hospitalization['geocoded_state'] = hospitalization['geocoded_state'].fillna("")
# and merge the weekly and dailies
hospitalization = hospitalization.merge(hospitalization_dailies, how='outer')
# the keep='last' here keeps just the daily data when both duplicate weekly and daily data exist
hospitalization = hospitalization.drop_duplicates(subset=['date', 'state'], keep='last', ignore_index=True)
# select a subset of columns to include in the output
hospitalization = hospitalization[[
    'state',
    'date', 
    'inpatient_beds_used_covid', 
    'total_adult_patients_hospitalized_confirmed_and_suspected_covid',
    'total_adult_patients_hospitalized_confirmed_covid',
    'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid',
    'total_pediatric_patients_hospitalized_confirmed_covid',
    'staffed_icu_adult_patients_confirmed_and_suspected_covid',
    'staffed_icu_adult_patients_confirmed_covid',
    'previous_day_admission_adult_covid_confirmed', 
    'previous_day_admission_adult_covid_suspected',
    'previous_day_admission_pediatric_covid_confirmed',
    'previous_day_admission_pediatric_covid_suspected',
    'inpatient_beds_used_covid_coverage',
    'total_adult_patients_hospitalized_confirmed_and_suspected_covid_coverage',
    'staffed_icu_adult_patients_confirmed_and_suspected_covid_coverage',
    'previous_day_admission_adult_covid_confirmed_coverage',
    'previous_day_admission_adult_covid_confirmed_18-19',
    'previous_day_admission_adult_covid_confirmed_20-29',
    'previous_day_admission_adult_covid_confirmed_30-39',
    'previous_day_admission_adult_covid_confirmed_40-49',
    'previous_day_admission_adult_covid_confirmed_50-59',
    'previous_day_admission_adult_covid_confirmed_60-69',
    'previous_day_admission_adult_covid_confirmed_70-79',
    'previous_day_admission_adult_covid_confirmed_80+',
    'previous_day_admission_adult_covid_confirmed_unknown',
    'staffed_adult_icu_bed_occupancy',
    'total_staffed_adult_icu_beds',
    'deaths_covid',
    ]]
# HHS hospitalization data gets better on and after 7/15/20
hospitalization = hospitalization[hospitalization['date'] >= datetime.date(year=2020,month=7,day=15)]

# case/death data: pick a subset of columns and prepare to merge
case_death = case_death[
    ['submission_date', 'state', 'tot_cases', 'conf_cases', 'prob_cases', 'new_case', 'pnew_case', 'tot_death',
     'conf_death', 'prob_death', 'new_death', 'pnew_death']]
case_death = case_death.rename(columns={'submission_date': 'date'})
# sum NY and NYC case/death data into one row
if COMBINE_NY_NYC:
  # set all NYC rows to NY, group and sum the NY rows, and combine with the rest of the rows
  case_death.loc[case_death['state'] == 'NYC', ['state']] = 'NY'
  NY_combined = case_death[case_death['state'] == 'NY'].groupby(["date", "state"]).sum()
  case_death = NY_combined.reset_index().append(case_death[case_death['state'] != 'NY'])

# merge all the dataframes together into one big combination
combined = pd.merge(left=testing, right=hospitalization, on=['state', 'date'], how='outer')

case_death['date'] = pd.to_datetime(case_death['date'], format='%m/%d/%Y')
combined['date'] = pd.to_datetime(combined['date'], format='%Y-%m-%d')

combined = combined.merge(case_death, on=['state', 'date'], how='outer')

combined.sort_values(by=['date', 'state'], inplace=True, ignore_index=True)

In [ ]:
vax_df = pd.read_csv('https://data.cdc.gov/api/views/unsk-b7fc/rows.csv?accessType=DOWNLOAD', parse_dates=['Date'])
vax_df

In [ ]:
combined_vax_df = pd.merge(combined[['state','date','previous_day_admission_adult_covid_confirmed','previous_day_admission_pediatric_covid_confirmed','new_case','new_death']],
         vax_df[['Location','Date','Administered_Dose1_Recip_18PlusPop_Pct','Series_Complete_18PlusPop_Pct',
                 'Administered_Dose1_Pop_Pct','Series_Complete_Pop_Pct',
                 'Administered_Dose1_Recip','Series_Complete_Yes',
                 'Administered',
         ]],
         how='left',
         left_on=['state','date'],
         right_on=['Location','Date']
         )

combined_vax_df.drop(['Location','Date'], axis=1, inplace=True)
combined_vax_df['date_str'] = combined_vax_df['date'].astype('str')
combined_vax_df

In [ ]:
cpr_url = 'https://beta.healthdata.gov/api/views/gqxm-d9w9/files/6723effd-0325-4f98-bed5-1218c25a5ab6?download=true&filename=Community%20Profile%20Report%2020210708.xlsx'
cpr_df = pd.read_excel(cpr_url, sheet_name='States', header=1)
cpr_df

In [ ]:
combined_vax_df = pd.merge(combined_vax_df, cpr_df[['State Abbreviation','Population']], how='left', left_on='state', right_on='State Abbreviation')
combined_vax_df

In [ ]:
def calc_rollingavg(df, col, days=7):
  df = df.sort_values(['state','date'], ascending=True)
  for state in df['state'].unique():
    df.loc[df['state']==state,f'{col}_{days}davg'] = \
    df.loc[df['state']==state,f'{col}'].rolling(window=days, min_periods=6).mean()
  
  return df

# National overview of cases, vax administered, % of pop vaxxed

In [ ]:
combined_vax_df

In [ ]:
national_df = combined_vax_df.groupby(['date']).sum()[['new_case','previous_day_admission_adult_covid_confirmed','previous_day_admission_pediatric_covid_confirmed','Administered_Dose1_Recip','Series_Complete_Yes','Administered','Population']]
national_df['new_admissions_confirmed'] = national_df[['previous_day_admission_adult_covid_confirmed','previous_day_admission_pediatric_covid_confirmed']].sum(axis=1)
national_df['series_complete_percent'] = national_df['Series_Complete_Yes']/national_df['Population']
national_df['firstdose_percent'] = national_df['Administered_Dose1_Recip']/national_df['Population']
national_df.reset_index(inplace=True)

In [ ]:
national_df['new_doses'] = national_df['Administered'].diff()

In [ ]:
days=7
for col in ['new_case','new_admissions_confirmed','new_doses']:
    national_df.loc[:,f'{col}_{days}davg'] = national_df.loc[:,f'{col}'].rolling(window=days, min_periods=7).mean()

In [ ]:
national_df

In [ ]:
source = national_df[(national_df['date']>='2021-01-01')&(national_df['date']<now_est_date)]

base_national = alt.Chart(source).encode(
    x=alt.X('date:T', title=None, axis=alt.Axis(grid=False, tickCount=10)),
).properties(width=1100, height=150)

case_bars = base_national.mark_bar(opacity=0.8, width=1, color='#D67300').encode(
    y='new_case'
)

case_lines = base_national.mark_line(color='#D67300').encode(
    y='new_case_7davg'
).properties(title='New Cases by Day')

admits_bars = base_national.mark_bar(opacity=0.8, width=1, color='#D67300').encode(
    y='new_admissions_confirmed'
)

admits_lines = base_national.mark_line(color='orange').encode(
    y='new_admissions_confirmed_7davg'
).properties(title='New Hospital Admissions by Day')


vaxcomplete_bars = base_national.mark_bar(opacity=0.5, width=5, color='green').encode(
    y='series_complete_percent'
)

vaxstarted_bars = base_national.mark_bar(opacity=0.5, width=5, color='green').encode(
    y=alt.Y('firstdose_percent', axis=alt.Axis(format='.0%'), scale=alt.Scale(domain=[0,.6])),
).properties(title='% of USA Total Population Vaccinated')

vaxadmin_bars = base_national.mark_bar(opacity=0.5, width=1, color='teal').encode(
    y='new_doses:Q'
)

vaxadmin_lines = base_national.mark_line(color='teal').encode(
    y='new_doses_7davg:Q'
).properties(title='New Vaccine Doses Administered by Day')

viz_1 = ((case_bars+case_lines) & (vaxadmin_bars+vaxadmin_lines) & (vaxcomplete_bars+vaxstarted_bars)).configure_title(
    # titleAngle=0, 
    dx=-55,
    dy=-35,
    orient='none',
    align='left'
)

viz_1

# Cases, Admits, Deaths per 100k by State timeseries line plots



In [ ]:
combined_vax_df['confirmed_admits'] = combined_vax_df['previous_day_admission_adult_covid_confirmed']+combined_vax_df['previous_day_admission_pediatric_covid_confirmed']

In [ ]:
source_2 = combined_vax_df[['date','state','new_case','new_death','confirmed_admits','Population']]

In [ ]:
source_2 = calc_rollingavg(source_2, 'new_case')
source_2 = calc_rollingavg(source_2, 'new_death')
source_2 = calc_rollingavg(source_2, 'confirmed_admits')

In [ ]:
for col in ['confirmed_admits_7davg','new_case_7davg','new_death_7davg']:
  source_2[f'{col}_per100k'] = source_2[f'{col}']/(source_2['Population']/1e5)

In [ ]:
source_2

In [ ]:
col_suffix = '_7davg_per100k'
axis_format = '.1f'
title_modifier = 'per 100k Population'

color_range = ['#FF6B00','#34840F','#4C78A8','#00AFA7','#C3359B']
color_domain = ['LA','TN','AK','FL','KY']
state_init = [{'state':s} for s in color_domain]

new_source = source_2[(source_2['date']>='2021-01-01')&(source_2['date']<now_est_date)
                                &(~source_2['state'].isin(['VI','GU','MP','AS','FSM','RMI',]))][['state','date',f'confirmed_admits{col_suffix}',f'new_case{col_suffix}', f'new_death{col_suffix}']]
new_source.replace(np.inf,np.nan, inplace=True)

select_states = alt.selection_multi(fields=['state'], bind='legend', init=state_init)

new_base = alt.Chart(new_source).encode(
    x='date:T',
    stroke=alt.condition(select_states, alt.Stroke('state:O',  legend=None,#alt.Legend(columns=2, symbolLimit=0, title='State'),  
                                                   scale=alt.Scale(range=color_range, domain=color_domain)), alt.value('lightgrey')),
    strokeWidth=alt.condition(select_states, alt.value(3), alt.value(0.3)),    
    tooltip=['state']
).properties(height=550, width=320)

# plot 1
new_cases = new_base.mark_line(opacity=0.8,).encode(
    y=alt.Y(f'new_case{col_suffix}', axis=alt.Axis(format=axis_format, tickCount=7, ), scale=alt.Scale(domain=[0,np.nanmax(new_source[f'new_case{col_suffix}'].values)])),
).interactive(bind_x=False).properties(title=f'New Cases, {title_modifier}')

# plot 2
new_admits = new_base.mark_line(opacity=0.8).encode(
    y=alt.Y(f'confirmed_admits{col_suffix}', axis=alt.Axis(format=axis_format, tickCount=7, ), scale=alt.Scale(domain=[0,new_source[f'confirmed_admits{col_suffix}'].max()]))
).interactive(bind_x=False).properties(title=f'New Hospitalizations, {title_modifier}')

# plot 3
new_deaths = new_base.mark_line(opacity=0.8,clip=True).encode(
    y=alt.Y(f'new_death{col_suffix}', axis=alt.Axis(format=axis_format, tickCount=7, ), scale=alt.Scale(domain=[0,new_source[f'new_death{col_suffix}'].max()])),
).interactive(bind_x=False).properties(title=f'New Deaths, {title_modifier}')

viz_2 = (new_cases | new_admits | new_deaths).add_selection(select_states).configure_title(
    # titleAngle=0, 
    dx=-35,
    dy=-35,
    orient='none',
    align='left'
)

viz_2

# Save all to drive

In [ ]:
# this is for saving altair charts to png and svg, based on https://colab.research.google.com/github/altair-viz/altair_saver/blob/master/AltairSaver.ipynb#scrollTo=ZiTDBCAM_Ni8
!pip install -q altair_saver
!npm install --silent vega-lite vega-cli canvas

In [ ]:
from pathlib import Path
from altair_saver import save

SAVE_PATH = Path('assets')
SAVE_PATH.mkdir(exist_ok=True)

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

In [ ]:
def assets_to_gdrive(folder_name, localdir_path = SAVE_PATH, parentdir_id='17Kx2uZbQv1r5U1M9x_OXS4lpMU5c6Ym8'):
  # search gdrive for snapshot folder and save assets there if it already exists. 
  folder_id = ''
  file_list = gdrive.ListFile({'q': f"'{parentdir_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed=false"}).GetList()
  for file1 in file_list:
      if file1['title'] == folder_name: 
        folder_id = file1['id']
        print(f'Found pre-existing gdrive folder named "{folder_name}" at',folder_id)
  # if not, create new folder
  if folder_id == '':
    folder = gdrive.CreateFile(metadata={'title': folder_name,
                                      'parents':[{'id': parentdir_id}],
                                      "mimeType": "application/vnd.google-apps.folder"
                                      })
    folder.Upload()
    folder_id = folder.get('id')
    print(f'Created new gdrive folder named "{folder_name}" at',folder_id)
  
  # upload all files within SAVE_PATH to snapshot folder
  for asset_file in localdir_path.iterdir():
    file_path = localdir_path/asset_file.name
    file1 = gdrive.CreateFile(metadata={'title':asset_file.name,
                                        'parents':[{'id': folder_id}],
                                        })
    file1.SetContentFile(file_path)
    file1.Upload()
    print('Saved file: ',asset_file.name)


In [ ]:
def save_vizassets(chart, save_path, filename, fmts=['html','json','png','svg','pdf']):
  for fmt in fmts:
    save(chart, f'{save_path}/{filename}.{fmt}')

In [ ]:
# put your stuff (i.e. dataframes, altair charts, input data files) to save here

In [ ]:
# examples:

# cpr_df.to_csv(SAVE_PATH/'cpr_df.csv')
# source_concat.to_csv(SAVE_PATH/'source_concat.csv')
# save_vizassets(state_vax_scatter, SAVE_PATH, f'state_vax_scatter_{now_utc_timestamp}')

combined_vax_df.to_csv(SAVE_PATH/'combined_vax_df.csv')
save_vizassets(viz_1, SAVE_PATH, f'viz_1_{now_utc_timestamp}')
save_vizassets(viz_2, SAVE_PATH, f'viz_2_{now_utc_timestamp}')

In [ ]:
# get the colab filename
from requests import get
nb_name = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'].replace('.ipynb','')
nb_id = get('http://172.28.0.2:9000/api/sessions').json()[0]['notebook']['path'].replace('fileId=','')

print(SAVE_PATH, nb_name, now_utc_timestamp, nb_id)

# create a snapshot of this currently running notebook and save to SAVE_PATH
downloaded_nb = gdrive.CreateFile({'id':nb_id})   # replace the id with id of file you want to access
downloaded_nb.GetContentFile(SAVE_PATH/f'{nb_name}_{now_utc_timestamp}.ipynb')

In [ ]:
# upload everything to gdrive
assets_to_gdrive(folder_name=f'{nb_name}_{now_utc_timestamp}')